In [12]:
import os
import scipy as sp
import rsmf
import matplotlib
from matplotlib import pyplot as plt
from postProcessing.importData import *


#Insert name here
NAME = "64cl_u_s3" 
# first type 64cl_u_s1
# second type 64cl_u_s3
#################


PATH = "./simulations/" + NAME
if(os.path.isdir(NAME)):
    print("Enter a valid file name")
else:
    PATH += ".h5"
    data_0 = dataProcessing(PATH)


b'float'
SELF CONSISTENT [GAUGE, HARTREE]  ==> [True, False]
The current simulation does not export HoppingFields
Note: Magnetic free energy density not added to free energy density because defined on plaquettes


In [13]:
NAME = "64cl_v_s3" #11-vortex case 6422JO-1.70c12-1.h5 ; 7-vortex case #4822W-1, 11-vortex case 6422JO-1.70c19-1.h5 ; 4822J1, 4822J, 4822JL-1, 4822JL-10, 64MJO-1.70c19.h5
# first type - 64cl_v, 64cl_11, 64_cl_18, 64cl_LV, 64cl_LV18
# second type - 64cl_v_s3, 64cl_s_18, 64cl_s_11, 64cl_s_7, 64cl_s_3, 64cl_s_3-1, 64cl_s_3-1-1
# third type - 64cl_sv_2-2, 64cl_sv_18, 64cl_sv_11, 64cl_sv_7, 64cl_sv_LV_11, 64cl_sv_LV_5
# fourth type 64cl_sv_4
PATH = "./simulations/" + NAME
if(os.path.isdir(NAME)):
    print("Enter a valid file name")
else:
    PATH += ".h5"
    data = dataProcessing(PATH)

b'float'
SELF CONSISTENT [GAUGE, HARTREE]  ==> [True, False]
The current simulation does not export HoppingFields
Note: Magnetic free energy density not added to free energy density because defined on plaquettes


In [16]:
fmt = rsmf.setup(r"\documentclass[aps, prb, superscriptaddress, twocolumn, letterpaper, 10pt, amsfonts, amsmath, amssymb, groupedaddress, longbibliography]{revtex4-2}")

In [20]:
edge = 2
cp = 37
row = 34
d_1 = (np.abs(data_0.D_1) - np.abs(data.D_1))[row, cp:-edge]
d_2 = (np.abs(data_0.D_2) - np.abs(data.D_2))[row, cp:-edge]
f_b = (np.abs(data.B))[row, cp:-edge+1]

In [21]:
def fit_f(x, a1, c1, a2, c2):
    return a1 * np.exp(-c1 * x) + a2 * np.exp(-c2 * x)

In [22]:
def cost_function(args, x1, y1, x2, y2):
    a1, a2, b1, b2, c1, c2 = args
    res_y_1 = a1 * np.exp(-c1 * x1) + a2 * np.exp(-c2 * x1)
    res_y_2 = b1 * np.exp(-c1 * x2) + b2 * np.exp(-c2 * x2)
    return np.sum((res_y_1 - y1) ** 2 / y1 ** 2) + np.sum((res_y_2 - y2) ** 2 / y2 ** 2)

In [23]:
def fit_B(x, a1, c1):
    return a1 * np.exp(-c1 * x)

In [41]:
x = np.linspace(0, 0.5, len(d_1))

In [42]:
opt = sp.optimize.minimize(cost_function, (0.02, 0.3, 0.1, 0.07, 12.2, 20.3), args=(x, d_1, x, d_2))

In [43]:
opt_B = sp.optimize.curve_fit(fit_B, x, f_b)

In [44]:
def sum_e(x1, a1, a2, c1, c2):
    return a1 * np.exp(-c1 * x1) + a2 * np.exp(-c2 * x1)

In [45]:
a1, a2, b1, b2, c1, c2 = opt.x

In [46]:
start = 0
fin = -1

In [54]:
import matplotlib.transforms as mtransforms

In [61]:
fig = fmt.figure()
gs = fig.add_gridspec(3,2,width_ratios=[3,1])
ax1= fig.add_subplot(gs[:, 0])
ax3= fig.add_subplot(gs[0, 1])
ax4= fig.add_subplot(gs[1, 1])
ax5= fig.add_subplot(gs[2, 1])

ax=plt.gca()
color = next(ax._get_lines.prop_cycler)['color']
ax1.set_xlabel(r'$r/L$')
ax1.set_ylabel(r'$\left(\left| \Delta \left(\infty\right) \right| - \left| \Delta \left(r\right) \right|\right)^2$')
ax1.set_yscale('log')
ax1.scatter(x[start:fin], d_1[start:fin] ** 2, color=color)
ax1.plot(x[start:fin], sum_e(x[start:fin], a1, a2, c1, c2) ** 2, label = r'$\Delta_1$', color=color)

color = next(ax._get_lines.prop_cycler)['color']
ax1.scatter(x[start:fin], d_2[start:fin] ** 2, color=color)
ax1.plot(x[start:fin], sum_e(x[start:fin], b1, b2, c1, c2) ** 2, label = r'$\Delta_2$', color=color)
ax1.tick_params(axis='y')

color = next(ax._get_lines.prop_cycler)['color']
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.set_ylabel(r'$B$')  # we already handled the x-label with ax1
ax2.scatter(x[start:fin], f_b[start:fin], color=color)
ax2.plot(x[start:fin], fit_B(x[start:fin], *opt_B[0]), label = r'$B$', color=color)
ax2.set_yscale('log')
#ax1.scatter(dist / 48, ((energies - e_v - 1.001 * energies_b) / (e_v - e_u)), label = 'total')
ax3.imshow(np.abs(data.D_1))
ax3.text(0.0, 1.0, r'$\Delta_1$', verticalalignment='top', bbox=dict(facecolor='0.7', edgecolor='1', pad=3.0))
ax4.imshow(np.abs(data.D_2))
ax4.text(0.0, 1.0, r'$\Delta_2$', verticalalignment='top', bbox=dict(facecolor='1', edgecolor='0', pad=3.0))
ax5.imshow(data.B)
ax5.text(0.0, 1.0, r'$B$', verticalalignment='top', bbox=dict(facecolor='0.7', edgecolor='0', pad=3.0))

ax3.xaxis.set_visible(False)
ax3.yaxis.set_visible(False)
ax4.xaxis.set_visible(False)
ax4.yaxis.set_visible(False)
ax5.yaxis.set_visible(False)

fig.tight_layout() 
ax1.legend(loc=3)
ax2.legend(loc=1)

plt.subplots_adjust(hspace=0.05)
plt.savefig('images/single_vortex.pdf')
plt.show()

/tmp/ipykernel_1581310/1848679588.py:47: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
